In [1]:
# Define the path to the results directory
directory = '/data2/jasonl/sandcastles/distinguisher/results'

In [2]:
import pandas as pd
import os
from matplotlib import pyplot as plt

def combine_files_by_base_name(prefix, distinguisher):
    """
    Combines files in a directory based on their base name, excluding certain files.

    Parameters:
        directory (str): Path to the directory containing files to combine.

    Returns:
        dict: A dictionary where keys are base names and values are combined DataFrames.
    """
    file_groups = {}

    for filename in os.listdir(directory):
        if filename.split('_')[0] != prefix or filename.split('_')[-1].split('.')[0] != distinguisher:
            continue
        base_name = '_'.join(filename.split('_part')[0].split('_')[:-1])
        if base_name not in file_groups:
            file_groups[base_name] = []
        file_groups[base_name].append(filename)

    combined_dfs = {}
    for base_name, files in file_groups.items():
        combined_df = pd.DataFrame()

        for filename in sorted(files):  # Ensure files are processed in order of parts
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)

        combined_dfs[base_name] = combined_df

    return combined_dfs

# Helper function to separate attacks based on step_num reset
def separate_attacks(df, length=10000):
    attacks = []
    current_attack = []
    
    for idx, row in df.iterrows():
        # Start a new attack if the step_num resets
        if idx > 0 and (row['Num'] != df.loc[idx - 1, 'Num'] + 1 or row['Origin'] != df.loc[idx - 1, 'Origin']):
            attacks.append(pd.DataFrame(current_attack))
            current_attack = []        

        current_attack.append(row)
    
    # Append the last attack
    if current_attack:
        attacks.append(pd.DataFrame(current_attack))
    
    return attacks

def add_entropy_column(dfs):
    """
    Takes a list of DataFrames, checks if the length of the list is a multiple of 10,
    and adds an 'entropy' column to each DataFrame, containing the remainder modulo 10 of the DataFrame's position in the list.

    Parameters:
        dfs (list of pd.DataFrame): List of DataFrames.

    Returns:
        list of pd.DataFrame: The updated list of DataFrames with the added 'entropy' column.

    Raises:
        ValueError: If the length of the list is not a multiple of 10.
    """
    if len(dfs) % 10 != 0:
        raise ValueError("The length of the list must be a multiple of 10.")

    for i, df in enumerate(dfs):
        df['entropy'] = i % 10

    return dfs

def process_and_add_entropy(file_path, separate_attacks, add_entropy_column):
    """
    Streamlines the processing of a CSV file: reads it, extracts domain from filename,
    adds a domain column, separates attacks, and adds an entropy column.

    Parameters:
        file_path (str): Path to the CSV file.
        separate_attacks (function): Function to separate attacks from the DataFrame.
        add_entropy_column (function): Function to add an entropy column to the list of DataFrames.

    Returns:
        list of pd.DataFrame: List of processed DataFrames with entropy column added.
    """
    domain = os.path.basename(file_path).split('_')[-2]
    df = pd.read_csv(file_path)
    df['domain'] = domain
    attacks = separate_attacks(df)
    return add_entropy_column(attacks)

def check_unique_values(df, columns):
    """
    Checks whether specified columns in a DataFrame have a single unique value and extracts this value.

    Parameters:
        df (pd.DataFrame): The DataFrame to check.
        columns (list of str): List of column names to check.

    Returns:
        dict: A dictionary with column names as keys and their unique value if single, else None.
    """
    unique_values = {}
    for col in columns:
        unique_vals = df[col].unique()
        if len(unique_vals) == 1:
            unique_values[col] = unique_vals[0]
        else:
            unique_values[col] = None
    return unique_values

def return_correct_flipped_correct(df, prefix=""):
    df.loc[:, f'{prefix}correct'] = (df['Origin'] == df[f'{prefix}choice']).astype(float)
    df.loc[:, f'{prefix}flipped_correct'] = (df['Origin'] == df[f'{prefix}flipped_choice']).astype(float)
    df.loc[:, f'{prefix}avg_correct'] = (df[f'{prefix}correct'] + df[f'{prefix}flipped_correct']) / 2
    return df[f'{prefix}correct'].mean(), df[f'{prefix}flipped_correct'].mean(),df[f'{prefix}avg_correct']. mean()

def get_attacks(prefix, distinguisher):
    combined_dataframes = combine_files_by_base_name(prefix, distinguisher)
    all_attacks = []
    all_num_attacks = []
    for base_name, combined_df in combined_dataframes.items():
        combined_df['domain'] = base_name.split('_')[-1]  # Infer domain from base name
        attacks = separate_attacks(combined_df)
        # attacks = add_entropy_column(attacks)

        num_attacks = [attack[attack['Num'] >= 100] for attack in attacks]

        all_attacks.extend(attacks)
        all_num_attacks.extend(num_attacks)
    return all_attacks, all_num_attacks


In [3]:
from collections import defaultdict
def print_stats(prefix, distinguisher):
    all_attacks, all_num_attacks = get_attacks(prefix, distinguisher)
    counter = defaultdict(int)
    countertail = defaultdict(int)
    for attack in all_attacks:
        return_correct_flipped_correct(attack)
        for i in range(len(attack)):
            counter[attack['avg_correct'].values[i]] += 1
        countertail[attack['avg_correct'].values[-1]] += 1
    score = sum(k * v for k, v in counter.items()) / sum(counter.values())
    print(prefix, distinguisher, score, list(counter.items()))

prefix = "llama3.1-8B"
for distinguisher in ["SimpleDistinguisher", "SimpleGPT", "SimplestGPT", "AggressiveSimple", "LogicGPT", "LogicGPTrev", "LogicSimple", "LogicSimplest"]:
    print_stats(prefix, distinguisher)
prefix = "gpt"
for distinguisher in ["SimplestGPT"]:
    print_stats(prefix, distinguisher)
prefix = "llama3.1-70B"
for distinguisher in ["SimpleDistinguisher", "AggressiveSimple", "LogicGPT", "LogicSimplest", "SimplestGPT"]:
    print_stats(prefix, distinguisher)

llama3.1-8B SimpleDistinguisher 0.955 [(1.0, 824), (0.0, 5), (0.5, 71)]
llama3.1-8B SimpleGPT 0.5133333333333333 [(1.0, 184), (0.5, 556), (0.0, 160)]
llama3.1-8B SimplestGPT 0.5672222222222222 [(1.0, 191), (0.5, 639), (0.0, 70)]
llama3.1-8B AggressiveSimple 0.8894444444444445 [(1.0, 714), (0.5, 173), (0.0, 13)]
llama3.1-8B LogicGPT 0.8633333333333333 [(1.0, 662), (0.5, 230), (0.0, 8)]
llama3.1-8B LogicGPTrev 0.5044444444444445 [(0.5, 892), (1.0, 8)]
llama3.1-8B LogicSimple 0.6916666666666667 [(0.5, 555), (1.0, 345)]
llama3.1-8B LogicSimplest 0.76 [(1.0, 468), (0.5, 432)]
gpt SimplestGPT 0.9477777777777778 [(1.0, 806), (0.5, 94)]
llama3.1-70B SimpleDistinguisher 1.0 [(1.0, 900)]
llama3.1-70B AggressiveSimple 0.9972222222222222 [(1.0, 895), (0.5, 5)]
llama3.1-70B LogicGPT 0.9972222222222222 [(1.0, 895), (0.5, 5)]
llama3.1-70B LogicSimplest 0.9972222222222222 [(1.0, 895), (0.5, 5)]
llama3.1-70B SimplestGPT 0.6972222222222222 [(0.5, 487), (1.0, 384), (0.0, 29)]


In [4]:
prefix = "llama3.1-70B"
distinguisher = "AggressiveSimple"
all_attacks, all_num_attacks = get_attacks(prefix, distinguisher)

# Combine original DataFrames for concatenated view (optional)
long = pd.concat(all_attacks, ignore_index=True)
len(long)

900

In [5]:
data = []

for attack in all_num_attacks:    
    correct, flipped_correct, avg_correct = return_correct_flipped_correct(attack)
    d = check_unique_values(attack, ['domain', 'entropy'])

    data.append({
        'domain' : d['domain'],
        'entropy' : d['entropy'],
        'correct': correct,
        'flipped_correct': flipped_correct,
        'avg_correct': avg_correct
    })
    
data_df = pd.DataFrame(data)

In [6]:
def group_df(df, group_col, stat):
    grouped_df = (
        df.groupby(group_col)
          .agg({
            'correct': stat,
            'flipped_correct': stat,
            'avg_correct': stat,
          })
          .reset_index()
    )
    return grouped_df

print(group_df(data_df, 'domain', 'mean'))
print(group_df(data_df, 'entropy', 'std'))

  domain   correct  flipped_correct  avg_correct
0   test  0.998462         0.993846     0.996154
   entropy   correct  flipped_correct  avg_correct
0      1.0  0.000000         0.047140     0.023570
1      2.0  0.000000         0.000000     0.000000
2      3.0  0.000000         0.108781     0.054391
3      4.0  0.000000         0.000000     0.000000
4      5.0  0.000000         0.000000     0.000000
5      6.0  0.000000         0.000000     0.000000
6      7.0  0.000000         0.000000     0.000000
7      8.0  0.000000         0.000000     0.000000
8      9.0  0.000000         0.000000     0.000000
9     10.0  0.070711         0.000000     0.035355


In [7]:
# Create a list to store the attacks and their corresponding avg values
attack_avg_list = []

for i, attack in enumerate(all_attacks):
    attack['id'] = i
    correct, flipped, avg = return_correct_flipped_correct(attack)
    attack_avg_list.append((attack, avg))  # Append the attack and its avg value as a tuple

# Sort the list by the avg value (second element of the tuple)
sorted_attacks = sorted(attack_avg_list, key=lambda x: x[1])

# Extract the sorted attacks and their averages separately if needed
sorted_attacks_list = [item[0] for item in sorted_attacks]
sorted_avgs = [item[1] for item in sorted_attacks]

# # Optionally, print the sorted averages for verification
# for attack, avg in sorted_attacks:
#     print(f"Attack: {attack}, Avg: {avg}")


In [8]:
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, IntSlider

# Function to plot cumulative avg progression for a given attack index
def plot_cumulative_avg(attack_index):
    # Get the attack for the given index
    attack = sorted_attacks_list[attack_index]

    avg_values = []
    
    # Iterate over the rows of the DataFrame cumulatively
    for i in range(1, len(attack) + 1):
        # Select the first `i` rows
        subset = attack.iloc[:i]
        
        # Apply `return_correct_flipped_correct` to the subset
        correct, flipped, avg = return_correct_flipped_correct(subset)
        
        # Store the avg value
        avg_values.append(avg)

    # Plot the graph
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(avg_values) + 1), avg_values, marker='o')
    plt.title(f'Cumulative Avg Progression for Attack Index {attack_index} (id={attack["id"].iloc[0]})')
    plt.xlabel('Number of Rows')
    plt.ylabel('Average')
    plt.grid(True)
    plt.show()

# Function to plot rolling avg progression for a given attack index
def plot_rolling_avg(attack_index, window_size=10):
    # Get the attack for the given index
    attack = sorted_attacks_list[attack_index]

    # Calculate rolling average
    rolling_avg = attack['avg_correct'].rolling(window=window_size).mean()

    # Plot the graph
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(rolling_avg) + 1), rolling_avg, marker='o')
    plt.title(f'Rolling Avg Progression for Attack Index {attack_index} (Window Size: {window_size}), (id={attack["id"].iloc[0]})')
    plt.xlabel('Number of Rows')
    plt.ylabel('Rolling Average')
    plt.grid(True)
    plt.show()


# Create an interactive slider to select attack index
def plot_selected(attack_index, plot_type, window_size=10):
    if plot_type == 'cumulative':
        plot_cumulative_avg(attack_index)
    elif plot_type == 'rolling':
        plot_rolling_avg(attack_index, window_size)

interact(
    plot_selected,
    attack_index=IntSlider(min=0, max=len(sorted_attacks_list) - 1, step=1, value=100),
    plot_type={
        'Cumulative Average': 'cumulative',
        'Rolling Average': 'rolling'
    },
    window_size=IntSlider(min=1, max=50, step=1, value=10)
)

interactive(children=(IntSlider(value=100, description='attack_index', max=179), Dropdown(description='plot_ty…

<function __main__.plot_selected(attack_index, plot_type, window_size=10)>

In [9]:
sorted_attacks_list[0][['Origin', 'choice', 'flipped_choice', 'correct', 'flipped_correct', 'avg_correct']].tail(5)

,Origin,choice,flipped_choice,correct,flipped_correct,avg_correct
80,A,A,B,1.0,0.0,0.5
81,A,A,A,1.0,1.0,1.0
82,A,A,A,1.0,1.0,1.0
83,A,A,B,1.0,0.0,0.5
84,A,A,A,1.0,1.0,1.0


In [10]:
from IPython.display import display, HTML

def show(df):
    # Display the DataFrame in a scrollable container
    display(HTML('<div style="max-height: 300px; overflow-y: scroll;">{}</div>'.format(
        df.to_html(index=False)
    )))

In [11]:
show(sorted_attacks_list[0][['Origin', 'prompt', 'P', 'origin_A', 'origin_B']].tail(2))

In [12]:
# convert the list of attacks to a single dataframe
first_rows = [attack.iloc[0].to_frame().T for attack in sorted_attacks_list]
attacks_df = pd.concat(first_rows, ignore_index=True)
# add sorted_avgs as a column to the dataframe
attacks_df['avg_correct'] = sorted_avgs
attacks_df[['o_str', 'w_str', 'm_str', 'avg_correct']][:20]

,o_str,w_str,m_str,avg_correct
0,InternLMOracle,distinguisher,SpanMutator,0.8
1,InternLMOracle,distinguisher,SpanMutator,0.9
2,InternLMOracle,distinguisher,WordMutator,0.9
3,InternLMOracle,distinguisher,WordMutator,0.9
4,InternLMOracle,distinguisher,SpanMutator,1.0
5,InternLMOracle,distinguisher,SpanMutator,1.0
6,InternLMOracle,distinguisher,SpanMutator,1.0
7,InternLMOracle,distinguisher,SpanMutator,1.0
8,InternLMOracle,distinguisher,SpanMutator,1.0
9,InternLMOracle,distinguisher,SpanMutator,1.0


In [13]:
i = 0
origin_A = sorted_attacks_list[i]['origin_A'].values[0]
origin_B = sorted_attacks_list[i]['origin_B'].values[0]
P = sorted_attacks_list[i]['P'].values[-1]

print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_A}
___________________________________________________________
Original response B: {origin_B}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

Modified text: **Title: Local Teachers' Strike Leaves School District at An Educational Impasse**

**City, State - Date** — The local teachers' union went on strike Monday morning, paralyzing the school district, as negotiations between the representatives of the union and school board regarding educator salaries, class size, and educational resources for students continue. This strike represents an especially critical moment in the ongoing struggle to improve educational conditions across the city, as educators, parents, and administrators alike clash with the district over limited financial resources.

At the heart of these discussions, educators have demanded wage and benefit increases, a mandated reduction in class size, and additional school resources. The teachers' union has indicated these demands are vital in both enhancing the educational experience for thousands of students and ensuring educators are able to support a costly living environment.
 "We are striking not only for 

In [14]:
print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_B}
___________________________________________________________
Original response B: {origin_A}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

Modified text: **Title: Local Teachers' Strike Leaves School District at An Educational Impasse**

**City, State - Date** — The local teachers' union went on strike Monday morning, paralyzing the school district, as negotiations between the representatives of the union and school board regarding educator salaries, class size, and educational resources for students continue. This strike represents an especially critical moment in the ongoing struggle to improve educational conditions across the city, as educators, parents, and administrators alike clash with the district over limited financial resources.

At the heart of these discussions, educators have demanded wage and benefit increases, a mandated reduction in class size, and additional school resources. The teachers' union has indicated these demands are vital in both enhancing the educational experience for thousands of students and ensuring educators are able to support a costly living environment.
 "We are striking not only for 

In [15]:
show(sorted_attacks_list[0].tail(1))